In [1]:
from PIL import Image as PilImage
import os
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from IPython.display import Image, display
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras import metrics
from sklearn.metrics import precision_score
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler

In [2]:
# def resize_image(input_image_path, output_image_path, size):
#     original_image = Image.open(input_image_path)
#     width, height = original_image.size
#     print(f"The original image size is {width} wide x {height} tall")

#     resized_image = original_image.resize(size)
#     width, height = resized_image.size
#     print(f"The resized image size is {width} wide x {height} tall")
#     resized_image.show()
#     resized_image.save(output_image_path)

In [3]:
# resize_image('C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Coachwhip.jpg', 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Coachwhip_resized.jpg', (950, 660))

In [4]:
# resize_image('C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Pinesnake.jpg', 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Pinesnake_resized.jpg', (950, 660))

In [5]:
# resize_image('C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Short-tailed Kingsnake.jpg', 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Short-tailed Kingsnake_resized.jpg', (950, 660))

In [6]:
image_dir = 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos'

# # Get list of image file names
# image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# # Create DataFrame
# df = pd.DataFrame(image_files, columns=['Image'])

# # Function to display image
# def display_image(filename):
#     return display(Image(filename=os.path.join(image_dir, filename)))

# # Apply function to DataFrame
# df['Image'].apply(display_image)

In [7]:
all_images = set(f for f in os.listdir(image_dir) if f.endswith('.jpg'))

In [8]:
venomous_snakes = ["Florida Cottonmouth.jpg", "Florida Cottonmouth 1.jpg", "Dusky Pygmy Rattlesnake.jpg", "Dusky Pygmy Rattlesnake 1.jpg", "Eastern Diamond-backed Rattlesnake.jpg", "Eastern Diamond-backed Rattlesnake 1.jpg",  "Harlequin Coralsnake.jpg", "Harlequin Coralsnake 1.jpg", "Eastern Copperhead.jpg", "Eastern Copperhead 1.jpg", "Timber Rattlesnake.jpg", "Timber Rattlesnake 1.jpg"]
venomous_snakes = set(venomous_snakes)
non_venomous_snakes = all_images - venomous_snakes

In [9]:
venomous_snakes = list(venomous_snakes)
non_venomous_snakes = list(non_venomous_snakes)



df = pd.DataFrame({
    'Venomous': pd.Series(venomous_snakes),
    'Non-venomous': pd.Series(non_venomous_snakes)
})

print(df)

                                    Venomous                    Non-venomous
0                 Harlequin Coralsnake 1.jpg            Brown Watersnake.jpg
1              Dusky Pygmy Rattlesnake 1.jpg           Eastern Kingsnake.jpg
2   Eastern Diamond-backed Rattlesnake 1.jpg     Eastern Hog-nosed Snake.jpg
3                   Harlequin Coralsnake.jpg              Burmese Python.jpg
4                     Eastern Copperhead.jpg               Rainbow Snake.jpg
..                                       ...                             ...
91                                       NaN     Florida Crowned Snake 1.jpg
92                                       NaN      Rim Rock Crowned Snake.jpg
93                                       NaN  Southeastern Crowned Snake.jpg
94                                       NaN        Striped Swampsnake 1.jpg
95                                       NaN             Red Cornsnake 1.jpg

[96 rows x 2 columns]


In [10]:
venomous_snakes

['Harlequin Coralsnake 1.jpg',
 'Dusky Pygmy Rattlesnake 1.jpg',
 'Eastern Diamond-backed Rattlesnake 1.jpg',
 'Harlequin Coralsnake.jpg',
 'Eastern Copperhead.jpg',
 'Dusky Pygmy Rattlesnake.jpg',
 'Florida Cottonmouth.jpg',
 'Timber Rattlesnake.jpg',
 'Timber Rattlesnake 1.jpg',
 'Eastern Diamond-backed Rattlesnake.jpg',
 'Eastern Copperhead 1.jpg',
 'Florida Cottonmouth 1.jpg']

In [11]:
df

,Venomous,Non-venomous
0,Harlequin Coralsnake 1.jpg,Brown Watersnake.jpg
1,Dusky Pygmy Rattlesnake 1.jpg,Eastern Kingsnake.jpg
2,Eastern Diamond-backed Rattlesnake 1.jpg,Eastern Hog-nosed Snake.jpg
3,Harlequin Coralsnake.jpg,Burmese Python.jpg
4,Eastern Copperhead.jpg,Rainbow Snake.jpg
...,...,...
91,NaN,Florida Crowned Snake 1.jpg
92,NaN,Rim Rock Crowned Snake.jpg
93,NaN,Southeastern Crowned Snake.jpg
94,NaN,Striped Swampsnake 1.jpg


In [12]:
# Get a list of all files in that directory
all_files = os.listdir(image_dir)

# Filter the list for files ending in '.jpg'
image_paths = [os.path.join(image_dir, file) for file in all_files if file.endswith('.jpg')]

In [13]:
# Initialize an empty list to store the numpy arrays
images = []

for path in image_paths:
    # Open the image file
    with PilImage.open(path) as img:
        # Convert the image data to a numpy array
        img_array = np.array(img)
        
        # Append the image array to the list
        images.append(img_array)

# Convert the list of image arrays into a single numpy array
images = np.array(images)

print(images.shape)
# Images, Height, Width, Color Channels

(108, 600, 900, 3)


In [14]:
labels = []

for path in image_paths:
    # Get the name of the snake from the path
    snake_name = os.path.basename(path)  # this gets the filename with the extension
    
    # Check if the snake is venomous
    if snake_name in venomous_snakes:
        # If it is, append 1 to the labels list
        labels.append(1)
    else:
        # If it's not, append 0 to the labels list
        labels.append(0)

# Convert the list of labels into a numpy array
labels = np.array(labels)
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [15]:


# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [16]:


# # Create the model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(660, 950, 3)))  # change input shape to match your images
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))  # 1 because we have 2 classes


In [17]:
# # Compile the model
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=[metrics.Precision()])

In [18]:
# # Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# # Early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# # Calculate class weights
# class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

# # Convert class weights to a dictionary to pass it to the fit function
# class_weight_dict = dict(enumerate(class_weights))

# # Define the undersampling method
# undersample = RandomUnderSampler(sampling_strategy='majority')

# # Flatten the image data
# X_train_flat = X_train.reshape(X_train.shape[0], -1)

# # Apply the undersampling method
# X_under_flat, y_under = undersample.fit_resample(X_train_flat, y_train)

# # Reshape the undersampled data back to its original shape
# X_under = X_under_flat.reshape(-1, 660, 950, 3)

# # Fit the model using the undersampled data
# model.fit(datagen.flow(X_under, y_under, batch_size=16),
#           epochs=20,
#           validation_data=(X_test, y_test),
#           callbacks=[early_stopping],
#           class_weight=class_weight_dict)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = dict(enumerate(weights))

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(600, 900, 3)))  # change input shape to match your images
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) 

model.compile(optimizer='adam', loss='binary_crossentropy')

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64, callbacks=[early_stopping])


Epoch 1/20
2/2 [==============================] - 3s 1s/step - loss: 1607.8937 - val_loss: 2063.4902
Epoch 2/20
2/2 [==============================] - 3s 1s/step - loss: 5627.0693 - val_loss: 684.3905
Epoch 3/20
2/2 [==============================] - 3s 1s/step - loss: 3770.8481 - val_loss: 99.6855
Epoch 4/20
2/2 [==============================] - 3s 1s/step - loss: 2507.7490 - val_loss: 143.9402
Epoch 5/20
2/2 [==============================] - 3s 1s/step - loss: 516.1849 - val_loss: 53.0130
Epoch 6/20
2/2 [==============================] - 3s 1s/step - loss: 332.3927 - val_loss: 3.7340
Epoch 7/20
2/2 [==============================] - 3s 1s/step - loss: 71.8119 - val_loss: 0.3065
Epoch 8/20
2/2 [==============================] - 3s 1s/step - loss: 9.0214 - val_loss: 0.5782
Epoch 9/20
2/2 [==============================] - 3s 1s/step - loss: 3.2444 - val_loss: 1.2479
Epoch 10/20
2/2 [==============================] - 3s 1s/step - loss: 3.8611 - val_loss: 0.6821


In [22]:
loss = model.evaluate(X_test, y_test, verbose=0)

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)  

f1 = f1_score(y_test, y_pred, average='weighted')
print(f'Test loss: {loss:.4f}, Test F1 score: {f1*100:.2f}%')


Test loss: 0.6821, Test F1 score: 86.58%
